#### Load data

In [110]:
with open('log/http.log','r') as InputData:
    APPdata = InputData.readlines()
    APPdata = [i.strip().split('\t') for i in APPdata]
    APPdata = list(filter(lambda x:'#' not in x[0],APPdata))

In [111]:
with open('log/IIIHTTP3.log','r') as InputData2:
    Testdata = InputData2.readlines()
    Testdata = [i.strip().split('\t') for i in Testdata]
    Testdata = list(filter(lambda x:'#' not in x[0],Testdata))

In [112]:
def similarity(test_data,whiteList_data):
    score = 0
    for key in fingerprint_keys_index:
        for key_index in fingerprint_keys_index[key]:
            if test_data[key_index] in whiteList_data[key]:
                score += 1
    #if score > 1:
    #    WhiteListUpdate(test_data,whiteList_data)
    #else:
    #    outgoingInfo()
    return score

In [113]:
def WhiteListUpdate(data):
    pass

In [114]:
def outgoingInfo(label,data):
    req_len = data[13]
    res_len = data[14]
    #label = data[12]
    if req_len > trainedList[label]['request_len']:
        print(req_len,trainedList[label]['request_len'])
        #return 
    elif res_len > trainedList[label]['respond_len']:
        print(res_len,trainedList[label]['respond_len'])
    else:
        return 

In [146]:
def average_data_len(label,req_data,res_data,aveReq,aveRes,num):
    average_req = 0
    average_res = 0
    if req_data != 0:
        num[0] += 1
        average_req = (aveReq *(num[0] - 1) + req_data)/num[0]
    else:
        average_req = aveReq
        
    if res_data != 0:
        num[1] += 1
        average_res = (aveRes *(num[1] - 1) + res_data)/num[1]
    else:
        average_res = aveRes

    return average_req,average_res,num

#### Training the white list fingerprint

In [147]:
fingerprint_keys_index={'ip':[4],'host':[8],'userAgent':[12]}
trainedList = {}
for index in range(len(APPdata)):
    if APPdata[index][12].strip() != "-":
        label = APPdata[index][12].replace('\'','').strip()
        if label not in trainedList:
            tmp_dict = {'ip':[APPdata[index][4].replace('\'','').strip()],
                        'host':[APPdata[index][8].replace('\'','').strip()],
                        'userAgent':[label],
                       'num':[0,0]}
            trainedList.setdefault(label)
            trainedList[label] = tmp_dict
            if int(APPdata[index][13].replace('\'','').strip()) != 0:
                trainedList[label]['request_len'] = float(APPdata[index][13].replace('\'','').strip())
            else:
                trainedList[label]['request_len'] = 0

            if int(APPdata[index][14].replace('\'','').strip()) != 0:
                trainedList[label]['respond_len'] = float(APPdata[index][14].replace('\'','').strip())
            else:
                trainedList[label]['respond_len'] = 0
        else:
            for key in fingerprint_keys_index:
                for key_index in fingerprint_keys_index[key]:
                    tmp_APPdata = APPdata[index][key_index].replace('\'','').strip()
                    if  tmp_APPdata not in trainedList[label][key]:
                        trainedList[label].setdefault(key,[]).append(tmp_APPdata)
        trainedList[label]['request_len'],trainedList[label]['respond_len'],trainedList[label]['num'] = average_data_len(label,
                                                                                                                         float(APPdata[index][13]),
                                                                                                                         float(APPdata[index][14]),
                                                                                                                         trainedList[label]['request_len'],
                                                                                                                         trainedList[label]['respond_len'],
                                                                                                                         trainedList[label]['num'])


In [148]:
print(trainedList['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/11.0.1 Safari/604.3.5'])

{'ip': ['140.122.185.5', '203.69.81.73', '198.35.26.112', '131.252.208.59', '158.195.17.250', '195.251.63.103', '210.69.124.205', '151.139.237.113', '210.69.124.221', '210.71.254.242', '216.58.200.234', '104.19.192.102', '203.66.68.6', '203.66.65.19', '210.71.253.81', '211.76.247.16', '103.227.176.9', '172.217.160.110', '31.13.87.5', '216.58.200.35'], 'host': ['www.csie.ntnu.edu.tw', 'www.firstpost.com', 'upload.wikimedia.org', 'web.cecs.pdx.edu', 'ipsc.ksp.sk', 'sigmod17contest.athenarc.gr', 'www.judicial.gov.tw', 'code.jquery.com', 'jirs.judicial.gov.tw', 'www.ewant.org', 'fonts.googleapis.com', 'cdnjs.cloudflare.com', 'cou02.ewant.org', 'cou01.ewant.org', 'cou03.ewant.org', '211.76.247.16', 'www.springresort.com.tw', 'translate.google.com', 'connect.facebook.net', 'staticxx.facebook.com', 'fonts.gstatic.com'], 'userAgent': ['Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/11.0.1 Safari/604.3.5'], 'num': [0, 262], 'request_len': 0, 'res

驗證

In [149]:
sum_len = 0
counter = 0
for i in APPdata:
    if i[12] =='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/604.3.5 (KHTML, like Gecko) Version/11.0.1 Safari/604.3.5':
        if int(i[14]) != 0:
            sum_len += int(i[14])
            counter += 1
        else:
            print('hi')
print(counter)
print(sum_len/counter)

hi
hi
262
103924.86259541985


In [150]:
toBeDetermined = []
NoHost =[]
for test_index in range(len(Testdata)):
    similarityScore = 0
    if Testdata[test_index][8] == '-':
        NoHost.append(Testdata[test_index])
    else:
        if Testdata[test_index][12] in trainedList:
            whiteList = trainedList[Testdata[test_index][12]]
            similarityScore = similarity(Testdata[test_index],whiteList)
        else:
            max_score = 0
            likelihoodAPP = []
            for label in trainedList:
                similarityScore = similarity(Testdata[test_index],trainedList[label])
                if similarityScore > max_score:
                    likelihoodAPP = Testdata[test_index]
                    max_score = similarityScore
                elif (similarityScore > 0) and (similarityScore < max_score):
                    likelihoodAPP.append(Testdata[test_index])
            if max_score == 0:
                toBeDetermined.append(Testdata[test_index])
            #print(similarityScore,Testdata[test_index][12],likelihoodAPP)